In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import pandas as pd
import pdb
# !pip install prophet
import prophet
# !pip install utils
import utils

COUNTY-LEVEL DATASETS

In [ ]:
data_path = '/content/drive/MyDrive/STATS307/Data/'
cases_df = pd.read_csv(data_path + 'covid-us-filtered.csv')
mobility_df = pd.read_csv(data_path + 'mobilitycountymice.csv')

Mobility

In [ ]:
mobility_df.drop(['country_region_code', 'country_region'], axis=1, inplace=True)
mobility_df['county'] = mobility_df['sub_region_2'].apply(lambda x: x[:-7])
mobility_df['county-state'] = mobility_df['county'] + ', ' + mobility_df['sub_region_1'] + ', US'
mobility_df.rename(columns={'retail_and_recreation_percent_change_from_baseline':'retail_and_recreation',\
                    'grocery_and_pharmacy_percent_change_from_baseline':'grocery_and_pharmacy',\
                    'parks_percent_change_from_baseline':'parks',\
                    'transit_stations_percent_change_from_baseline':'transit_stations',\
                    'workplaces_percent_change_from_baseline':'workplaces',\
                    'residential_percent_change_from_baseline':'residential'}, inplace=True)
mobility_df['date'] = pd.to_datetime(mobility_df['date'])

In [ ]:
mobility_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9730 entries, 0 to 9729
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   sub_region_1           9730 non-null   object        
 1   sub_region_2           9730 non-null   object        
 2   date                   9730 non-null   datetime64[ns]
 3   retail_and_recreation  9730 non-null   int64         
 4   grocery_and_pharmacy   9730 non-null   int64         
 5   parks                  9730 non-null   int64         
 6   transit_stations       9730 non-null   int64         
 7   workplaces             9730 non-null   int64         
 8   residential            9730 non-null   int64         
 9   county                 9730 non-null   object        
 10  county-state           9730 non-null   object        
dtypes: datetime64[ns](1), int64(6), object(4)
memory usage: 836.3+ KB


In [ ]:
mobility_df['county-state'].unique()

array(['Los Angeles, California, US', 'New York, New York, US',
       'Suffolk, Massachusetts, US', 'Miami-Dade, Florida, US',
       'Cook, Illinois, US', 'Santa Barbara, California, US',
       'Orange, New York, US', 'Franklin, Massachusetts, US',
       'Highlands, Florida, US', 'Whiteside, Illinois, US'], dtype=object)

Filter to common dates for all counties

In [ ]:
locs = cases_df.Combined_Key.values
print(locs)
dates = []
for key in locs:
  dates.append(set(mobility_df[mobility_df['county-state']==key]['date']))

dates = sorted(list(set.intersection(*dates)))

['Los Angeles, California, US' 'Santa Barbara, California, US'
 'Highlands, Florida, US' 'Miami-Dade, Florida, US' 'Cook, Illinois, US'
 'Whiteside, Illinois, US' 'Franklin, Massachusetts, US'
 'Suffolk, Massachusetts, US' 'New York, New York, US'
 'Orange, New York, US']


In [ ]:
# dates

In [ ]:
TRAIN_START, TRAIN_DUR = 107, 466-107
TEST_END = -288

In [ ]:
print('before: {}'.format(mobility_df.shape))
mobility_df = mobility_df[mobility_df['date'].isin(dates[TRAIN_START-1: TEST_END])] #TRAIN_START-1 to adjust for .diff()
print('after: {}'.format(mobility_df.shape))

before: (9730, 11)
after: (5740, 11)


In [ ]:
mobility_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5740 entries, 106 to 9441
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   sub_region_1           5740 non-null   object        
 1   sub_region_2           5740 non-null   object        
 2   date                   5740 non-null   datetime64[ns]
 3   retail_and_recreation  5740 non-null   int64         
 4   grocery_and_pharmacy   5740 non-null   int64         
 5   parks                  5740 non-null   int64         
 6   transit_stations       5740 non-null   int64         
 7   workplaces             5740 non-null   int64         
 8   residential            5740 non-null   int64         
 9   county                 5740 non-null   object        
 10  county-state           5740 non-null   object        
dtypes: datetime64[ns](1), int64(6), object(4)
memory usage: 538.1+ KB


In [ ]:
locs = cases_df.Combined_Key.values
urbans = ['Los Angeles, California, US', 'New York, New York, US', 
          'Suffolk, Massachusetts, US', 'Miami-Dade, Florida, US', 'Cook, Illinois, US']
states = list(mobility_df['sub_region_1'].unique())
X_dict = {key: None for key in locs}
for key, _ in X_dict.items():
  X_dict[key] = mobility_df[mobility_df['county-state']==key]

  # Difference
  # X_dict[key].iloc[:,3:-2] = X_dict[key].iloc[:,3:-2].diff(axis=0)
  X_dict[key] = X_dict[key].iloc[1:]
  X_dict[key]['state'] = X_dict[key]['sub_region_1']
  X_dict[key].drop(['sub_region_1', 'sub_region_2'], axis=1, inplace=True)
  # Normalize
  X_dict[key].iloc[:, 1:-3]=(X_dict[key].iloc[:, 1:-3]-X_dict[key].iloc[:, 1:-3].mean())/X_dict[key].iloc[:, 1:-3].std()
  # Encode location info
  # X_dict[key]['urban'] = 0
  # if key in urbans:
    # X_dict[key]['urban'] = 1
  
  # state = X_dict[key]['state'].values[0]
  # for s in states: 
  #  X_dict[key][s] = 0
   # if s == state:
   #   X_dict[key][s] = 1

#  X_dict[key]['dayofweek'] = X_dict[key]['date'].dt.dayofweek
#  X_dict[key]['month'] = X_dict[key]['date'].dt.month
#  X_dict[key]['year'] = X_dict[key]['date'].dt.year
#  X_dict[key]['day'] = X_dict[key]['date'].dt.day
#  X_dict[key]['year_mod'] = (X_dict[key]['year'] - X_dict[key]['year'].min()) / (X_dict[key]['year'].max() - X_dict[key]['year'].min())

  # Remove unnecessary columns
  X_dict[key].drop(['county', 'county-state', 'state'], axis=1, inplace=True)

In [ ]:
for key in X_dict:
  print(key)
  X_dict[key].info()

Los Angeles, California, US
<class 'pandas.core.frame.DataFrame'>
Int64Index: 573 entries, 107 to 685
Data columns (total 7 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   date                   573 non-null    datetime64[ns]
 1   retail_and_recreation  573 non-null    float64       
 2   grocery_and_pharmacy   573 non-null    float64       
 3   parks                  573 non-null    float64       
 4   transit_stations       573 non-null    float64       
 5   workplaces             573 non-null    float64       
 6   residential            573 non-null    float64       
dtypes: datetime64[ns](1), float64(6)
memory usage: 35.8 KB
Santa Barbara, California, US
<class 'pandas.core.frame.DataFrame'>
Int64Index: 573 entries, 4977 to 5555
Data columns (total 7 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   date          

In [ ]:
# X = np.dstack([v.iloc[:,1:] for v in X_dict.values()])
# X.shape

Y: filter by available dates

In [ ]:
Y = cases_df.drop(labels=['State', 'Country_Region', 'County','Combined_Key'], axis=1)\
    .iloc[:, 24:-28].diff(axis=1).T.iloc[1:,:].reset_index()
Y['index'] = pd.to_datetime(Y['index'])
Y = Y[Y['index'].isin(dates[TRAIN_START: TEST_END])]

In [ ]:
Y['index']

106   2020-06-01
107   2020-06-02
108   2020-06-03
109   2020-06-04
110   2020-06-05
         ...    
680   2021-12-27
681   2021-12-28
682   2021-12-29
683   2021-12-30
684   2021-12-31
Name: index, Length: 573, dtype: datetime64[ns]

In [ ]:
columns = cases_df['Combined_Key'].values
rename_cols = {v: columns[v] for v in range(10)}
Y = Y.rename(columns=rename_cols)
#Y = Y.drop('index', axis=1).to_numpy() # Drop index(=date) variable

In [ ]:
Y

,index,"Los Angeles, California, US","Santa Barbara, California, US","Highlands, Florida, US","Miami-Dade, Florida, US","Cook, Illinois, US","Whiteside, Illinois, US","Franklin, Massachusetts, US","Suffolk, Massachusetts, US","New York, New York, US","Orange, New York, US"
106,2020-06-01,1010.0,10.0,1.0,139.0,570.0,2.0,0.0,69.0,74.0,16.0
107,2020-06-02,1208.0,33.0,2.0,85.0,1178.0,0.0,0.0,68.0,40.0,27.0
108,2020-06-03,1042.0,19.0,2.0,232.0,531.0,2.0,0.0,42.0,26.0,11.0
109,2020-06-04,1431.0,29.0,4.0,323.0,509.0,3.0,3.0,78.0,79.0,11.0
110,2020-06-05,1376.0,46.0,3.0,277.0,631.0,2.0,0.0,38.0,72.0,13.0
...,...,...,...,...,...,...,...,...,...,...,...
680,2021-12-27,7409.0,686.0,0.0,0.0,29718.0,79.0,92.0,2082.0,3201.0,317.0
681,2021-12-28,9451.0,304.0,0.0,0.0,9443.0,64.0,103.0,1242.0,5372.0,461.0
682,2021-12-29,16483.0,530.0,0.0,0.0,9766.0,59.0,77.0,2354.0,8086.0,1512.0
683,2021-12-30,20169.0,577.0,0.0,0.0,12931.0,116.0,100.0,3231.0,8112.0,1598.0


In [ ]:
# delete FL and MA 
del X_dict['Cook, Illinois, US']
del X_dict['Whiteside, Illinois, US']
del X_dict['Suffolk, Massachusetts, US']
del X_dict['Franklin, Massachusetts, US']



In [ ]:
X_dict.keys()

dict_keys(['Los Angeles, California, US', 'Santa Barbara, California, US', 'Highlands, Florida, US', 'Miami-Dade, Florida, US', 'New York, New York, US', 'Orange, New York, US'])

SET UP SARIMA

In [ ]:
!pip install pmdarima
import pmdarima as pm
import sklearn
from sklearn.metrics import mean_squared_error as mse
from statsmodels.tsa.stattools import adfuller
from statsmodels.tools.eval_measures import rmse, aic
from statsmodels.tsa.stattools import kpss
from statsmodels.tsa.stattools import grangercausalitytests

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
X_dict

{'Los Angeles, California, US':           date  retail_and_recreation  grocery_and_pharmacy     parks  \
 107 2020-06-01              -2.627705             -2.163466 -1.495582   
 108 2020-06-02              -2.274177             -0.107635 -0.903169   
 109 2020-06-03              -1.567121              1.200621  0.112398   
 110 2020-06-04              -1.449279              0.826834  0.281659   
 111 2020-06-05              -1.567121              0.639940 -0.141494   
 ..         ...                    ...                   ...       ...   
 681 2021-12-27               0.554045             -0.855210 -2.257257   
 682 2021-12-28               1.261101              0.826834 -0.649277   
 683 2021-12-29              -0.035168             -0.855210 -2.934301   
 684 2021-12-30               0.082675             -0.107635 -3.611346   
 685 2021-12-31               0.082675              3.630240 -0.903169   
 
      transit_stations  workplaces  residential  
 107         -1.394513   -0.7

# PROPHET MODEL

In [ ]:
from prophet import Prophet
import utils

In [ ]:
## data pre-processing for one city
cmodel = {}
weeklypreds = {}
weekly_test_MSE = {}



for key in X_dict:
  X_county = X_dict[key]
  Y_county = Y.loc[:, Y.columns.isin([key, 'index'])]
    # select X based on county
  X_county_train = X_county[(X_county['date'] >= '2020-06-01') & (X_county['date'] < '2021-06-01')].iloc[:,:] # 1 here for arima
  X_county_test = X_county[(X_county['date'] < '2021-06-15') & (X_county['date'] >= '2021-06-01')].iloc[:,:] 
    # select Y based on county
  Y_county_train = Y_county[(Y_county['index'] >= '2020-06-01') & (Y_county['index'] < '2021-06-01')][key]
  Y_county_test = Y_county[(Y_county['index'] < '2021-06-15') & (Y_county['index'] >= '2021-06-01')][key]
    # reset indices
  Y_county_train = Y_county_train.reset_index(drop=True) 
  X_county_train  = X_county_train.reset_index(drop=True)
  Y_county_test = Y_county_test.reset_index(drop=True) 
  X_county_test = X_county_test.reset_index(drop=True) 
    # combine train and test
  traindf = X_county_train.copy()
  testdf = X_county_test.copy()
  # deal with nan's and 0's in Y
  Y_county_train = Y_county_train.replace(0, 1e-5) # add noise to 0 so they don't become infinite
  Y_county_train[Y_county_train < 0] = 1e-5
  Y_county_train = np.log(Y_county_train)
  Y_county_train = Y_county_train.fillna(0)
  # deal with nan's and 0's in Y_test
  Y_county_test = Y_county_test.replace(0, 1e-5) # add noise to 0 so they don't become infinite
  Y_county_test = np.log(Y_county_test)
  Y_county_test = Y_county_test.fillna(0)
  # merge X and Y into one and label date as ds and y
  traindf['y'] = Y_county_train
  testdf['y'] = Y_county_test
  traindf.rename(columns = {'date':'ds'}, inplace = True)
  testdf.rename(columns = {'date':'ds'}, inplace = True)
  # data formatting
  testdf = testdf.drop(columns=['ds', 'y'])
  predtrain = traindf.drop(columns=['ds', 'y'])
  fores = pd.concat([predtrain, testdf], axis = 0)
  fores = fores.reset_index(drop=True) 

  #MODEL weekly:
  m = Prophet(seasonality_mode = 'multiplicative', daily_seasonality=False, yearly_seasonality=False)
  # add each regression for traindf to model
  xreg = [] # store names to be able to easily add to test
  for column in traindf:
    if (column != 'y') & (column != 'ds'): # col name not y or ds
      m.add_regressor(column, mode='multiplicative')
      xreg.append(column)
  # run model
  m.fit(traindf)
  # make future + forecast
  future = m.make_future_dataframe(periods=len(testdf), freq='1D') # daily future w/o CI
  futures = pd.concat([future, fores], axis = 1)
  forecast = m.predict(futures)
  preds1 = forecast.tail(14)
  weeklypreds[key] = preds1
  preds = preds1['yhat']
  true = Y_county_test
  error = mse(true, preds)
  weekly_test_MSE[key] = error

DEBUG:cmdstanpy:input tempfile: /tmp/tmp1lp09orj/w3maf1to.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp1lp09orj/0wqb9gvq.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: 1
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.8/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=98913', 'data', 'file=/tmp/tmp1lp09orj/w3maf1to.json', 'init=/tmp/tmp1lp09orj/0wqb9gvq.json', 'output', 'file=/tmp/tmp1lp09orj/prophet_model9qz6tc0_/prophet_model-20221207033621.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
03:36:21 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
03:36:21 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
/usr/local/lib/python3.8/dist-packages/pandas/core/arraylike.py:364: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
DEBUG:cmdstanpy:input tempfile: /tmp/tmp1lp09orj/z9q7x292.json
DEBUG:cmdsta

In [ ]:
weeklypreds

{'Los Angeles, California, US':             ds     trend  yhat_lower  yhat_upper  trend_lower  trend_upper  \
 359 2021-06-01  5.794054    1.390489    7.357348     5.794054     5.794054   
 360 2021-06-02  5.786811    2.250715    8.017121     5.786811     5.786811   
 361 2021-06-03  5.779567    3.127445    9.051751     5.779567     5.779567   
 362 2021-06-04  5.772323    1.790623    7.583304     5.772323     5.772323   
 363 2021-06-05  5.765079    1.309344    7.153986     5.765079     5.765079   
 364 2021-06-06  5.757835    1.877859    8.038041     5.757835     5.757835   
 365 2021-06-07  5.750591    2.684820    8.348436     5.750591     5.750591   
 366 2021-06-08  5.743347    2.309362    8.239446     5.743347     5.743347   
 367 2021-06-09  5.736104    2.124577    8.237925     5.736104     5.736104   
 368 2021-06-10  5.728860    2.753525    8.457958     5.728860     5.728860   
 369 2021-06-11  5.721616    1.075126    6.891689     5.721616     5.721616   
 370 2021-06-12  5.71

In [ ]:
weekly_test_MSE

{'Los Angeles, California, US': 1.0909862081685056,
 'Santa Barbara, California, US': 0.5726163267268365,
 'Highlands, Florida, US': 107.70121916980683,
 'Miami-Dade, Florida, US': 225.61422883867547,
 'New York, New York, US': 0.7462233483795954,
 'Orange, New York, US': 6.14969666972825}

In [ ]:
## data pre-processing for one city
cmodel = {}
yearlypreds = {}
yearly_test_MSE = {}



for key in X_dict:
  X_county = X_dict[key]
  Y_county = Y.loc[:, Y.columns.isin([key, 'index'])]
    # select X based on county
  X_county_train = X_county[(X_county['date'] >= '2020-06-01') & (X_county['date'] < '2021-06-01')].iloc[:,:] # 1 here for arima
  X_county_test = X_county[(X_county['date'] < '2021-06-15') & (X_county['date'] >= '2021-06-01')].iloc[:,:] 
    # select Y based on county
  Y_county_train = Y_county[(Y_county['index'] >= '2020-06-01') & (Y_county['index'] < '2021-06-01')][key]
  Y_county_test = Y_county[(Y_county['index'] < '2021-06-15') & (Y_county['index'] >= '2021-06-01')][key]
    # reset indices
  Y_county_train = Y_county_train.reset_index(drop=True) 
  X_county_train  = X_county_train.reset_index(drop=True)
  Y_county_test = Y_county_test.reset_index(drop=True) 
  X_county_test = X_county_test.reset_index(drop=True) 
    # combine train and test
  traindf = X_county_train.copy()
  testdf = X_county_test.copy()
  # deal with nan's and 0's in Y
  Y_county_train = Y_county_train.replace(0, 1e-5) # add noise to 0 so they don't become infinite
  Y_county_train[Y_county_train < 0] = 1e-5
  Y_county_train = np.log(Y_county_train)
  Y_county_train = Y_county_train.fillna(0)
  # deal with nan's and 0's in Y_test
  Y_county_test = Y_county_test.replace(0, 1e-5) # add noise to 0 so they don't become infinite
  Y_county_test = np.log(Y_county_test)
  Y_county_test = Y_county_test.fillna(0)
  # merge X and Y into one and label date as ds and y
  traindf['y'] = Y_county_train
  testdf['y'] = Y_county_test
  traindf.rename(columns = {'date':'ds'}, inplace = True)
  testdf.rename(columns = {'date':'ds'}, inplace = True)
  # data formatting
  testdf = testdf.drop(columns=['ds', 'y'])
  predtrain = traindf.drop(columns=['ds', 'y'])
  fores = pd.concat([predtrain, testdf], axis = 0)
  fores = fores.reset_index(drop=True) 

  #MODEL weekly:
  m = Prophet(seasonality_mode = 'multiplicative', daily_seasonality=False, yearly_seasonality=True, weekly_seasonality = False)
  # add each regression for traindf to model
  xreg = [] # store names to be able to easily add to test
  for column in traindf:
    if (column != 'y') & (column != 'ds'): # col name not y or ds
      m.add_regressor(column, mode='multiplicative')
      xreg.append(column)
  # run model
  m.fit(traindf)
  # make future + forecast
  future = m.make_future_dataframe(periods=len(testdf), freq='1D') # daily future w/o CI
  futures = pd.concat([future, fores], axis = 1)
  forecast = m.predict(futures)
  preds1 = forecast.tail(14)
  yearlypreds[key] = preds1
  preds = preds1['yhat']
  true = Y_county_test
  error = mse(true, preds)
  yearly_test_MSE[key] = error

DEBUG:cmdstanpy:input tempfile: /tmp/tmp1lp09orj/hok_0zmr.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp1lp09orj/qcqqejgf.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: 1
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.8/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=80282', 'data', 'file=/tmp/tmp1lp09orj/hok_0zmr.json', 'init=/tmp/tmp1lp09orj/qcqqejgf.json', 'output', 'file=/tmp/tmp1lp09orj/prophet_modelza1k71ss/prophet_model-20221207033642.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
03:36:42 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
03:36:42 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
/usr/local/lib/python3.8/dist-packages/pandas/core/arraylike.py:364: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
DEBUG:cmdstanpy:input tempfile: /tmp/tmp1lp09orj/_1dsn5lm.json
DEBUG:cmdsta

In [ ]:
yearly_test_MSE

{'Los Angeles, California, US': 11.319346720652673,
 'Santa Barbara, California, US': 3.1245376988649194,
 'Highlands, Florida, US': 43.368214075934816,
 'Miami-Dade, Florida, US': 155.80249400030306,
 'New York, New York, US': 0.5612885242093503,
 'Orange, New York, US': 1.049258734826418}

In [ ]:
## data pre-processing for one city
cmodel = {}
dailypreds = {}
daily_test_MSE = {}



for key in X_dict:
  X_county = X_dict[key]
  Y_county = Y.loc[:, Y.columns.isin([key, 'index'])]
    # select X based on county
  X_county_train = X_county[(X_county['date'] >= '2020-06-01') & (X_county['date'] < '2021-06-01')].iloc[:,:] # 1 here for arima
  X_county_test = X_county[(X_county['date'] < '2021-06-15') & (X_county['date'] >= '2021-06-01')].iloc[:,:] 
    # select Y based on county
  Y_county_train = Y_county[(Y_county['index'] >= '2020-06-01') & (Y_county['index'] < '2021-06-01')][key]
  Y_county_test = Y_county[(Y_county['index'] < '2021-06-15') & (Y_county['index'] >= '2021-06-01')][key]
    # reset indices
  Y_county_train = Y_county_train.reset_index(drop=True) 
  X_county_train  = X_county_train.reset_index(drop=True)
  Y_county_test = Y_county_test.reset_index(drop=True) 
  X_county_test = X_county_test.reset_index(drop=True) 
    # combine train and test
  traindf = X_county_train.copy()
  testdf = X_county_test.copy()
  # deal with nan's and 0's in Y
  Y_county_train = Y_county_train.replace(0, 1e-5) # add noise to 0 so they don't become infinite
  Y_county_train[Y_county_train < 0] = 1e-5
  Y_county_train = np.log(Y_county_train)
  Y_county_train = Y_county_train.fillna(0)
  # deal with nan's and 0's in Y_test
  Y_county_test = Y_county_test.replace(0, 1e-5) # add noise to 0 so they don't become infinite
  Y_county_test = np.log(Y_county_test)
  Y_county_test = Y_county_test.fillna(0)
  # merge X and Y into one and label date as ds and y
  traindf['y'] = Y_county_train
  testdf['y'] = Y_county_test
  traindf.rename(columns = {'date':'ds'}, inplace = True)
  testdf.rename(columns = {'date':'ds'}, inplace = True)
  # data formatting
  testdf = testdf.drop(columns=['ds', 'y'])
  predtrain = traindf.drop(columns=['ds', 'y'])
  fores = pd.concat([predtrain, testdf], axis = 0)
  fores = fores.reset_index(drop=True) 

  #MODEL weekly:
  m = Prophet(seasonality_mode = 'multiplicative', daily_seasonality=True, yearly_seasonality=False, weekly_seasonality = False)
  # add each regression for traindf to model
  xreg = [] # store names to be able to easily add to test
  for column in traindf:
    if (column != 'y') & (column != 'ds'): # col name not y or ds
      m.add_regressor(column, mode='multiplicative')
      xreg.append(column)
  # run model
  m.fit(traindf)
  # make future + forecast
  future = m.make_future_dataframe(periods=len(testdf), freq='1D') # daily future w/o CI
  futures = pd.concat([future, fores], axis = 1)
  forecast = m.predict(futures)
  preds1 = forecast.tail(14)
  dailypreds[key] = preds1
  preds = preds1['yhat']
  true = Y_county_test
  error = mse(true, preds)
  daily_test_MSE[key] = error

DEBUG:cmdstanpy:input tempfile: /tmp/tmp1lp09orj/m0gwsfbw.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp1lp09orj/hcz5s74h.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: 1
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.8/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=61481', 'data', 'file=/tmp/tmp1lp09orj/m0gwsfbw.json', 'init=/tmp/tmp1lp09orj/hcz5s74h.json', 'output', 'file=/tmp/tmp1lp09orj/prophet_model_grqwh8z/prophet_model-20221207033740.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
03:37:40 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
03:37:40 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
/usr/local/lib/python3.8/dist-packages/pandas/core/arraylike.py:364: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
DEBUG:cmdstanpy:input tempfile: /tmp/tmp1lp09orj/_cosj42g.json
DEBUG:cmdsta

In [ ]:
daily_test_MSE

{'Los Angeles, California, US': 1.0644087229524823,
 'Santa Barbara, California, US': 0.6981029580347639,
 'Highlands, Florida, US': 109.30551136876593,
 'Miami-Dade, Florida, US': 221.51227630303995,
 'New York, New York, US': 0.822359237850064,
 'Orange, New York, US': 5.8505070134950845}

PLOTS

In [ ]:
fig1 = m.plot(forecast)

In [ ]:
fig2 = m.plot_components(forecast)